## prepare robots
1. turn on panda and indy
2. connect indy with conty (192.168.0.63 or 141.223.193.55)
3. reset indy
4. open panda desk (https://192.168.0.13/desk/) and open joints
5. hold the panda control handle
6. start panda repeater
```bash
ssh panda@192.168.0.172
roslaunch panda_ros_repeater joint_velocity_repeater.launch robot_ip:=192.168.0.13 load_gripper:=false
```
7. run cells below

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.panda, ((0,0,0),(0,0,0)),
                "{}/{}".format(PANDA_REPEATER_IP, PANDA_ROBOT_IP))]
              , connection_list=[False, False])

from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")
gscene = s_builder.create_gscene(crob)

from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.geometry.geotype import *
from pkg.planning.constraint.constraint_actor import Gripper2Tool
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="panda0_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)
pscene.create_binder(bname="grip0", gname="grip0", rname="panda0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))

from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.motion.moveit.moveit_py import JointState
mplan = MoveitPlanner(pscene)
mplan.update_gscene()

connection_list
[False, False]
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


In [10]:
from pkg.controller.repeater.panda_repeater import PandaRepeater
from pkg.controller.repeater.indy_repeater import indytraj_client
from pkg.project_config import *

class lpf3d:
    def __init__(self, tau, ts, stack=False):
        self.ts = ts
        self.tau = tau
        self.x = np.zeros(3)
        self.y = np.zeros(3)
        self.stack= stack
        self.log = []
        
    def push(self, x):
        self.x = np.array(x)
        
    def update(self):
        self.y = self.tau/(self.tau+self.ts)*self.y + self.ts/(self.tau+self.ts)*self.x
        if self.stack:
            self.log.append(self.y.copy())
        return self.y
    
import threading
import time

gtimer = GlobalTimer.instance()

class stop_holder:
    def __init__(self):
        self.stop = False
    
    def stop_now(self):
        self.stop = True

def thread_read_fun(indy, lpf, stopper, scale=1.0):
    stopper.stop = False
    with indy:
        zero_x = indy.get_ai(0)*scale
        zero_y = indy.get_ai(1)*scale
        while not stopper.stop:
            with gtimer.block("thread_read_fun"):
                sig_x = 0
                sig_y = np.clip((float(indy.get_ai(0))*scale-zero_x)/5000,-1, 1)
                sig_z = -np.clip((float(indy.get_ai(1))*scale-zero_y)/5000,-1, 1)
                lpf.push(np.array([sig_x, sig_y, sig_z]))
            
def panda_fun(panda, lpf, stopper, home_pose, d_max=0.1):
    stopper.stop = False
    q0 = home_pose
    jac_flat = mplan.planner.get_jacobian("panda0", JointState(7, *q0))
    jac_flat = [jac_flat[i_j] for i_j in range(len(jac_flat))]
    jac = np.reshape(jac_flat, (7, 6)).transpose()
    pinvj = np.linalg.pinv(jac)
    
    panda.move_joint_interpolated(q0)

    qcur = np.array(panda.get_qcur()) if q0 is None else q0    
    panda.reset(q0)
    try:
        while not stopper.stop:
            with gtimer.block("move"):
                sig_x, sig_y, sig_z = lpf.update()*d_max
                dp = [sig_x, sig_y, sig_z, 0, 0, 0]
                dq = np.matmul(pinvj, dp)
                panda.move_possible_joints_x4(q0 + dq)
    except Exception as e:
        print(e)
    finally:
        stopper.stop = True
        for _ in range(1000):
            lpf.push(np.array([0, 0, 0]))
            sig_x, sig_y, sig_z = lpf.update()*d_max
            dp = [sig_x, sig_y, sig_z, 0, 0, 0]
            dq = np.matmul(pinvj, dp)
            panda.move_possible_joints_x4(q0 + dq)

In [11]:
crob.reset_connection([True])
panda = crob.robot_dict['panda0']
indy = indytraj_client(INDY_IP, robot_name="NRMK-Indy7")

lpf = lpf3d(0.5, panda.rate_x4.sleep_dur.to_sec(), stack=True)
stopper = stop_holder()
gtimer.reset()

t_read = threading.Thread(target=thread_read_fun, args=(indy, lpf, stopper), kwargs={'scale':1.0})
t_read.start()

t_panda = threading.Thread(target=panda_fun, args=(panda, lpf, stopper, crob.home_pose), kwargs={'d_max':0.1})
t_panda.start()

connection_list
[True]
Connect: Server IP (192.168.0.63)


In [13]:
stopper.stop_now()